In [2]:
#Importação das bibliotecas para leitura do arquivo csv, classes para divisão em treinamento e testes, 
#algoritmos DecisionTreeClassifier e svm
import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

%matplotlib inline

#Leitura do arquivo para treinamento e testes. Este é o arquivo que foi modificado e que possui uma coluna booleana 
#bomIvestimento que foi definida com a ajuda de um especialista na área imobiliária
try:
    data = pd.read_csv("imoveis-treinamento-testes.csv", sep=";")
    print "Arquivo de treinamento e testes de imóveis carregado com sucesso!"
except:
    print "Dataset não pode ser carregado!"

    
#Troca 0.0 para False e 1.0 para True na coluna bomIvestimento
data['bomInvestimento'] = data['bomInvestimento'].map({0.0: False, 1.0: True}) 

#Preenche valores padrões para coluna de banheiros, vagas na garagem, dormitórios
#E metragem da casa
data['banheiros'] = data['banheiros'].fillna(1)
data['vagasGaragem'] = data['vagasGaragem'].fillna(0)
data['dormitorios'] = data['dormitorios'].fillna(1)
data['metragem'] = data['metragem'].fillna(0)

#Remove imóveis com preço sob consulta
data = data.drop(data[data.valor == "Sob consulta"].index)

#Mostro os tipos de dados de cada coluna para saber se a conversão foi feita corretamente
print data.dtypes

#Mostro algumas estatísticas do dataset
display(data.describe())

#Outras estatísticas sobre o conteúdo do dataset
print "Bons investimentos: " + str(len(data[data.bomInvestimento]))
print "Maus investimentos: " + str(len(data[data.bomInvestimento == False]))
print "Tamanho médio das casas com bom investimento: " + str(data[data.bomInvestimento].metragem.mean())
print "Tamanho médio das casas mal investimento: " + str(data[data.bomInvestimento == False].metragem.mean())
print "Número médio de dormitórios bom investimento: " + str(data[data.bomInvestimento].dormitorios.mean())
print "Número médio de dormitórios mal investimento: " + str(data[data.bomInvestimento == False].dormitorios.mean())

Arquivo de treinamento e testes de imóveis carregado com sucesso!
valor               object
bairro              object
obs                 object
tipo                object
dormitorios        float64
banheiros          float64
vagasGaragem       float64
metragem           float64
codigo              object
bomInvestimento       bool
dtype: object


,dormitorios,banheiros,vagasGaragem,metragem
count,249.000000,249.0,249.000000,249.000000
mean,2.650602,1.0,0.465863,143.201245
std,0.908300,0.0,0.499838,111.763697
min,1.000000,1.0,0.000000,0.000000
25%,2.000000,1.0,0.000000,74.480000
50%,3.000000,1.0,0.000000,113.940000
75%,3.000000,1.0,1.000000,182.000000
max,10.000000,1.0,1.000000,1300.000000


Bons investimentos: 17
Maus investimentos: 232
Tamanho médio das casas com bom investimento: 261.931176471
Tamanho médio das casas mal investimento: 134.501206897
Número médio de dormitórios bom investimento: 2.88235294118
Número médio de dormitórios mal investimento: 2.63362068966


In [36]:
# Fiz uma cópia do dataframe original removendo colunas não usadas para o fim de treinamento e testes
new_data = data.drop(['bomInvestimento', 'codigo', 'bairro', 'obs', 'tipo'], axis = 1)

#Divisão do dataset para fins de treinamento e testes
X_train, X_test, y_train, y_test = train_test_split(new_data, data['bomInvestimento'], test_size=0.30, random_state=158)

#Aplicando o algoritmo DecisionTreeClassifier e exibindo o score
dt = DecisionTreeClassifier(random_state=160)
dt.fit(X_train, y_train)
print "Score usando algoritmo DecisionTreeClassifier: " + str(dt.score(X_test, y_test))

aSvm = svm.SVC(random_state=160)
aSvm.fit(X_train, y_train)
print "Score usando algoritmo SVC: " + str(aSvm.score(X_test, y_test))

Score usando algoritmo DecisionTreeClassifier: 0.906666666667
Score usando algoritmo SVC: 0.946666666667


In [59]:
#Usando GridSearchCV para encontrar parâmetros melhores para os algoritmos de classificação
parametersDT = {
    'criterion': ['gini', 'entropy'], 
    'random_state': range(1,50),
    'min_samples_split': [2,3]
}
regressorDT = DecisionTreeClassifier()
clf2 = GridSearchCV(regressorDT, parametersDT, n_jobs=2)
clf2.fit(X_train, y_train)
score = clf2.best_estimator_.score(X_test, y_test)
print "Score do algoritmo DecisionTreeClassifier com GridSearchCV: " + str(score)
print "Melhores parâmetros encontrados para DecisionTreeClassifier"
print clf2.best_params_


parametersSVC = {
    'C': [0.01, 0.1, 0.5, 1] , 
    'kernel': ['sigmoid', 'rbf'], 
    'random_state': range(1,100)
}
regressorSVC = svm.SVC()
clf1 = GridSearchCV(regressorSVC, parametersSVC, n_jobs=2)
clf1.fit(X_train, y_train)
# TODO: Reportar a pontuação da previsão utilizando o conjunto de teste
score = clf1.best_estimator_.score(X_test, y_test)
print "Score do algoritmo SVC com GridSearchCV: " + str(score)
print "Melhores parâmetros encontrados para SVC"
print clf1.best_params_



Score do algoritmo DecisionTreeClassifier com GridSearchCV: 0.96
Melhores parâmetros encontrados para DecisionTreeClassifier
{'min_samples_split': 2, 'random_state': 3, 'criterion': 'gini'}
Score do algoritmo SVC com GridSearchCV: 0.946666666667
Melhores parâmetros encontrados para SVC
{'kernel': 'sigmoid', 'C': 0.01, 'random_state': 1}
